In [1]:
import sys
import time
import tensorflow as tf
import numpy as np
sys.path.insert(0, '../')
%load_ext autoreload
%autoreload 2

In [2]:
from seqmodel.bunch import Bunch
from seqmodel import model
from seqmodel import data

In [56]:
tf.reset_default_graph()
model_opt = model.seq2seq_model.BasicSeq2SeqModel.default_opt()
model_opt.output_mode = 'logit'
model_opt.loss_type = 'mse'
model_opt.decoder.rnn_opt.logit.out_vocab_size=1
model_opt.embedding.decoder_dim = 10
model_opt.embedding.encoder_dim = 10
model_opt.decoder.rnn_opt.rnn_cell.cell_opt.num_units = 10
model_opt.encoder.rnn_opt.rnn_cell.cell_opt.num_units = 10
model_opt.decoder.share.encoder_rnn_params = True
model_opt.decoder.share.encoder_embedding = True
model_opt.decoder.share.logit_weight_tying = False
model_fn = model.seq2seq_model.BasicSeq2SeqModel(model_opt)
seq2seq_model = model_fn(is_training=True)
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
g_v_pairs =  optimizer.compute_gradients(seq2seq_model.training_loss, tf.trainable_variables())
train_op = optimizer.apply_gradients(g_v_pairs)
for v in tf.trainable_variables():
    print("{}, {}".format(v.name, v.get_shape()))

model/shared_embedding:0, (15, 10)
model/shared_rnn/rnn/basic_lstm_cell/weights:0, (20, 40)
model/shared_rnn/rnn/basic_lstm_cell/biases:0, (40,)
model/decoder_rnn/logit_w:0, (1, 10)
model/decoder_rnn/logit_b:0, (1,)


In [57]:
vocab = data.Vocabulary.from_vocab_file('../data/tiny_copy/vocab.txt')
valid_iter = data.Seq2SeqIterator(vocab, vocab)
valid_iter.initialize('../data/tiny_copy/valid.txt')
valid_iter.init_batch(4)

In [71]:
sess_config = tf.ConfigProto(device_count={'GPU': 0})
sess = tf.Session(config = sess_config)
sess.run(tf.global_variables_initializer())
batch = valid_iter.next_batch()
_l = batch.labels
labels = _l._replace(decoder_label=np.zeros_like(_l.decoder_label).astype(np.float32))
batch = batch._replace(labels=labels)
state = None

In [90]:
prediction, state, info = seq2seq_model.predict(sess, batch.features)

In [73]:
eval_loss, state, info = seq2seq_model.evaluate(sess, batch)

In [88]:
eval_loss, train_loss, state, info = seq2seq_model.train(sess, batch, train_op)

In [89]:
eval_loss

0.00068569067

In [80]:
train_loss

0.001865093

In [91]:
prediction

array([[[  1.90190971e-04],
        [  3.09576094e-03],
        [  2.98764557e-03],
        [ -2.11770013e-02]],

       [[  1.41574778e-02],
        [  1.20249167e-02],
        [  9.36970115e-04],
        [ -1.20998360e-02]],

       [[ -2.01649964e-05],
        [ -3.27211246e-03],
        [  3.05108428e-02],
        [ -2.94937436e-02]],

       [[ -1.74429268e-03],
        [  1.00684799e-02],
        [  1.61853284e-02],
        [ -7.38602877e-03]],

       [[ -2.94423848e-03],
        [ -1.45296752e-03],
        [  2.28796974e-02],
        [ -4.68302816e-02]],

       [[ -4.68302816e-02],
        [ -1.60653889e-02],
        [  1.30661242e-02],
        [ -4.68302816e-02]],

       [[ -4.68302816e-02],
        [ -2.70652268e-02],
        [  1.37189440e-02],
        [ -4.68302816e-02]],

       [[ -4.68302816e-02],
        [  2.25233659e-03],
        [  1.51119828e-02],
        [ -4.68302816e-02]],

       [[ -4.68302816e-02],
        [ -7.91992992e-04],
        [  4.65014949e-03],
    